In [1]:
! pip install -U pip
! pip install -U torch==1.4.0
! pip install -U torchaudio==0.4.0
! pip install -U matplotlib==3.2.1
! pip install -U trains==0.15.0
! pip install -U pandas==1.0.4
! pip install -U numpy==1.18.4
! pip install -U tensorboard==2.2.1

Requirement already up-to-date: pip in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (20.1.1)
Requirement already up-to-date: torch==1.4.0 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (1.4.0)
Requirement already up-to-date: torchaudio==0.4.0 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (0.4.0)
Requirement already up-to-date: matplotlib==3.2.1 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (3.2.1)
Requirement already up-to-date: trains==0.15.0 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (0.15.0)
Requirement already up-to-date: pandas==1.0.4 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (1.0.4)
Requirement already up-to-date: numpy==1.18.4 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (1.18.4)


Requirement already up-to-date: tensorboard==2.2.1 in /home/sam/.pyenv/versions/3.7.4/envs/dan/lib/python3.7/site-packages (2.2.1)


In [2]:
import pandas as pd
import numpy as np
from pathlib2 import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import torchaudio

from trains import Task

%matplotlib inline

In [3]:
task = Task.init(project_name='Audio Example', task_name='audio classifier')
configuration_dict = {'number_of_epochs': 10, 'batch_size': 4, 'dropout': 0.25, 'base_lr': 0.001}
configuration_dict = task.connect(configuration_dict)  # enabling configuration override by trains
print(configuration_dict)  # printing actual configuration (after override in remote mode)

TRAINS Task: overwriting (reusing) task id=c55aabc92e434015a2c97d26b7ac3578
TRAINS results page: https://demoapp.trains.allegro.ai/projects/469622fa0a1842f9a90947bdc1773aea/experiments/c55aabc92e434015a2c97d26b7ac3578/output/log
{'number_of_epochs': 10, 'batch_size': 4, 'dropout': 0.25, 'base_lr': 0.001}


In [4]:
# Download UrbanSound8K dataset (https://urbansounddataset.weebly.com/urbansound8k.html)
path_to_UrbanSound8K = '/home/sam/Datasets/UrbanSound8K'

In [5]:
class UrbanSoundDataset(Dataset):
#rapper for the UrbanSound8K dataset
    def __init__(self, csv_path, file_path, folderList):
        self.file_path = file_path
        self.file_names = []
        self.labels = []
        self.folders = []
        
        #loop through the csv entries and only add entries from folders in the folder list
        csvData = pd.read_csv(csv_path)
        for i in range(0,len(csvData)):
            if csvData.iloc[i, 5] in folderList:
                self.file_names.append(csvData.iloc[i, 0])
                self.labels.append(csvData.iloc[i, 6])
                self.folders.append(csvData.iloc[i, 5])
        
    def __getitem__(self, index):
        #format the file path and load the file
        path = self.file_path / ("fold" + str(self.folders[index])) / self.file_names[index]
        sound, sample_rate = torchaudio.load(path, out = None, normalization = True)

        # UrbanSound8K uses two channels, this will convert them to one
        soundData = torch.mean(sound, dim=0, keepdim=True)
        
        #Make sure all files are the same size
        if soundData.numel() < 160000:
            fixedsize_data = torch.nn.functional.pad(soundData, (0, 160000 - soundData.numel()))
        else:
            fixedsize_data = soundData[0,:160000].reshape(1,160000)
        
        #downsample the audio
        downsample_data = fixedsize_data[::5]
        
        melspectogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate)
        melspectogram = melspectogram_transform(downsample_data)
        melspectogram_db = torchaudio.transforms.AmplitudeToDB()(melspectogram)

        return fixedsize_data, sample_rate, melspectogram_db, self.labels[index]
    
    def __len__(self):
        return len(self.file_names)


csv_path = Path(path_to_UrbanSound8K) / 'metadata' / 'UrbanSound8K.csv'
file_path = Path(path_to_UrbanSound8K) / 'audio'

train_set = UrbanSoundDataset(csv_path, file_path, range(1,10))
test_set = UrbanSoundDataset(csv_path, file_path, [10])
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

train_loader = torch.utils.data.DataLoader(train_set, batch_size = configuration_dict.get('batch_size', 4), 
                                           shuffle = True, pin_memory=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = configuration_dict.get('batch_size', 4), 
                                          shuffle = False, pin_memory=True, num_workers=1)

classes = ('air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 
           'gun_shot', 'jackhammer', 'siren', 'street_music')

Train set size: 7895
Test set size: 837


In [6]:
class CnnAudioNet(nn.Module):
    def __init__(self,NumClasses):
        super(CnnAudioNet,self).__init__()
        self.NumClasses = NumClasses
        self.Fc_features = 128
        self.C1 = nn.Conv2d(1,16,3)
        self.C2 = nn.Conv2d(16,32,3)
        self.C3 = nn.Conv2d(32,64,3)
        self.C4 = nn.Conv2d(64,128,3)
        
        self.BN1 = nn.BatchNorm2d(16)
        self.BN2 = nn.BatchNorm2d(32)
        self.BN3 = nn.BatchNorm2d(64)
        self.BN4 = nn.BatchNorm2d(128)
        self.maxpool1 = nn.MaxPool2d(2,2)        
        
        self.fc1 = nn.Linear(128*29*197,128)
        self.fc2 = nn.Linear(128,self.NumClasses )
        self.dropout = nn.Dropout(configuration_dict.get('dropout', 0.25))
    
    def forward(self,x):
        # add sequence of convolutional and max pooling layers
        x = F.relu(self.C1(x))
        x = self.maxpool1(F.relu(self.C2(x)))
        x = F.relu(self.C3(x))
        x = self.maxpool1(F.relu(self.C4(x)))
        # flatten image input
        x = x.view(-1,128*29*197)
        x =  F.relu(self.fc1(self.dropout(x)))
        x = self.fc2(self.dropout(x))
        return x
    
    
model = CnnAudioNet(10)

In [7]:
optimizer = optim.SGD(model.parameters(), lr = configuration_dict.get('base_lr', 0.001), momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)
criterion = nn.CrossEntropyLoss()

In [8]:
device = torch.cuda.current_device() if torch.cuda.is_available() else torch.device('cpu')
print('Device to use: {}'.format(device))
model.to(device)

Device to use: 0


CnnAudioNet(
  (C1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (C2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (C3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (C4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (BN1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=731264, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [9]:
tensorboard_writer = SummaryWriter('./tensorboard_logs')

In [10]:
import PIL
import io
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor


def plot_signal(signal, title, cmap=None):
    fig = plt.figure()
    if signal.ndim == 1:
        plt.plot(signal)
    else:
        plt.imshow(signal, cmap=cmap)    
    plt.title(title)
    
    plot_buf = io.BytesIO()
    plt.savefig(plot_buf, format='jpeg')
    plot_buf.seek(0)
    plt.close(fig)
    return ToTensor()(PIL.Image.open(plot_buf))

In [11]:
def train(model, epoch):
    model.train()
    for batch_idx, (sounds, sample_rate, inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        iteration = epoch * len(train_loader) + batch_idx
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(inputs), len(train_loader.dataset), 
                          100. * batch_idx / len(train_loader), loss))
            tensorboard_writer.add_scalar('training loss/loss', loss, iteration)
            tensorboard_writer.add_scalar('learning rate/lr', optimizer.param_groups[0]['lr'], iteration)
                
        
        if batch_idx % debug_interval == 0:    # report debug image every 500 mini-batches
            for n, (inp, pred, label) in enumerate(zip(inputs, predicted, labels)):
                series = 'label_{}_pred_{}'.format(classes[label.cpu()], classes[pred.cpu()])
                tensorboard_writer.add_image('Train MelSpectrogram samples/{}'.format(n), 
                                             plot_signal(inp.cpu().numpy().squeeze(), series, 'hot'), iteration)

In [12]:
def test(model, epoch):
    model.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for idx, (sounds, sample_rate, inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels)
            for i in range(len(inputs)):
                label = labels[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
        
            iteration = (epoch + 1) * len(train_loader)
            if idx % debug_interval == 0:    # report debug image every 100 mini-batches
                for n, (sound, inp, pred, label) in enumerate(zip(sounds, inputs, predicted, labels)):
                    series = 'label_{}_pred_{}'.format(classes[label.cpu()], classes[pred.cpu()])
                    print(iteration, sample_rate)
                    tensorboard_writer.add_audio('Test audio samples/{}'.format(n), 
                                                 sound, iteration, int(sample_rate[n]))
                    tensorboard_writer.add_image('Test MelSpectrogram samples/{}_{}'.format(idx, n), 
                                                 plot_signal(inp.cpu().numpy().squeeze(), series, 'hot'), iteration)

    total_accuracy = 100 * sum(class_correct)/sum(class_total)
    print('[Iteration {}] Accuracy on the {} test images: {}%\n'.format(epoch, sum(class_total), total_accuracy))
    tensorboard_writer.add_scalar('accuracy/total', total_accuracy, iteration)

In [13]:
log_interval = 100
debug_interval = 200
for epoch in range(configuration_dict.get('number_of_epochs', 10)):
    train(model, epoch)
    test(model, epoch)
    scheduler.step()

Train Epoch: 0 [0/7895 (0%)]	Loss: 2.314560
Train Epoch: 0 [400/7895 (5%)]	Loss: 2.281619
Train Epoch: 0 [800/7895 (10%)]	Loss: 2.479096
Train Epoch: 0 [1200/7895 (15%)]	Loss: 2.050175
Train Epoch: 0 [1600/7895 (20%)]	Loss: 1.865820
Train Epoch: 0 [2000/7895 (25%)]	Loss: 2.286825
Train Epoch: 0 [2400/7895 (30%)]	Loss: 2.025537
Train Epoch: 0 [2800/7895 (35%)]	Loss: 1.976062
Train Epoch: 0 [3200/7895 (41%)]	Loss: 2.277118
Train Epoch: 0 [3600/7895 (46%)]	Loss: 1.865269
Train Epoch: 0 [4000/7895 (51%)]	Loss: 1.828044
Train Epoch: 0 [4400/7895 (56%)]	Loss: 1.754791
Train Epoch: 0 [4800/7895 (61%)]	Loss: 1.316154
Train Epoch: 0 [5200/7895 (66%)]	Loss: 3.064564
Train Epoch: 0 [5600/7895 (71%)]	Loss: 1.080945
Train Epoch: 0 [6000/7895 (76%)]	Loss: 1.787307
Train Epoch: 0 [6400/7895 (81%)]	Loss: 1.813057
Train Epoch: 0 [6800/7895 (86%)]	Loss: 3.648711
Train Epoch: 0 [7200/7895 (91%)]	Loss: 2.033821
Train Epoch: 0 [7600/7895 (96%)]	Loss: 1.137187
1974 tensor([44100, 44100, 44100, 44100])
1974 

Train Epoch: 6 [400/7895 (5%)]	Loss: 0.160859
Train Epoch: 6 [800/7895 (10%)]	Loss: 0.664670
Train Epoch: 6 [1200/7895 (15%)]	Loss: 0.890169
Train Epoch: 6 [1600/7895 (20%)]	Loss: 0.584988
Train Epoch: 6 [2000/7895 (25%)]	Loss: 1.022784
Train Epoch: 6 [2400/7895 (30%)]	Loss: 0.345562
Train Epoch: 6 [2800/7895 (35%)]	Loss: 0.083229
Train Epoch: 6 [3200/7895 (41%)]	Loss: 0.281698
Train Epoch: 6 [3600/7895 (46%)]	Loss: 0.542019
Train Epoch: 6 [4000/7895 (51%)]	Loss: 0.412350
Train Epoch: 6 [4400/7895 (56%)]	Loss: 0.608566
Train Epoch: 6 [4800/7895 (61%)]	Loss: 0.855969
Train Epoch: 6 [5200/7895 (66%)]	Loss: 0.060554
Train Epoch: 6 [5600/7895 (71%)]	Loss: 0.490298
Train Epoch: 6 [6000/7895 (76%)]	Loss: 0.274956
Train Epoch: 6 [6400/7895 (81%)]	Loss: 0.688784
Train Epoch: 6 [6800/7895 (86%)]	Loss: 1.309547
Train Epoch: 6 [7200/7895 (91%)]	Loss: 0.731241
Train Epoch: 6 [7600/7895 (96%)]	Loss: 0.164598
13818 tensor([44100, 44100, 44100, 44100])
13818 tensor([44100, 44100, 44100, 44100])
13818